In [2]:
"mem" in {"mem": 1}

True

In [1]:
from agent.dqn import DQNAgent
import logging

# disable logging
logging.disable(logging.CRITICAL)

agent = DQNAgent(
    default_root_dir="./training-results/TRASH",
    # num_iterations=200,
    # replay_buffer_size=150,
    # warm_start=100,
    # batch_size=4,
    # capacity={"episodic": 16, "semantic": 16, "short": 10},
    # ddqn=True,
)
# agent.fill_replay_buffer()
agent.train()

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: T

Running on cpu


AttributeError: 'MemorySystems' object has no attribute 'short'